In [3]:
import os
import glob
import re

import pandas as pd


MODELS_PATH = './models/java-small-model'
results = []

In [25]:
models = {
        '2020_04_05__0.0001_0.0001_0.001__0026d470': 32,
        '2020_04_05__0.0001_0.0001_0.001__2ecc05a5': 22,
        '2020_04_05__0.0001_0.0001_0.001__e152caf0': 28,
        '2020_04_05__0_0_-1__2167eb3d': 37,
        '2020_04_05__0_0_-1__3cff1009': 19,
        '2020_04_05__0_0_-1__4990422c': 19}

In [12]:
models = {'2020_04_07__0_0_-2.0__subtoken_restricted': 24}

In [17]:
models = ['2020_04_09__0_0_-3.0__subtoken_nodes_restricted']

In [2]:
models = ['2020_04_05__0.0001_0.0001_0.001__e152caf0']

In [50]:
coeff_select = ['2020_04_07__0.1_0_-1__b718b076',
                '2020_04_07__0.01_0_-1__53cfd74e',
                '2020_04_07__0.001_0_-1__0f88a7ef',
                '2020_04_07__0.0001_0_-1__92371c27', 
                '2020_04_07__1e-05_0_-1__81cbb810',
                '2020_04_07__1e-06_0_-1__3866b871',
                '2020_04_07__0_0.1_-1__5810e829',
                '2020_04_07__0_0.01_-1__b303bfa9',
                '2020_04_07__0_0.001_-1__facca85d',
                '2020_04_07__0_0.0001_-1__35a3d3de',
                '2020_04_07__0_1e-05_-1__bfb47368',
                '2020_04_07__0_1e-06_-1__be492856',
                '2020_04_05__0_0_-1__2167eb3d']

In [51]:
models = coeff_select

# Best epoch and validation

In [52]:
non_decimal = re.compile(r'[^\d.]+')

results = []
for model in models:
    out_files = glob.glob(MODELS_PATH + '/' + model + "/*.out")
    if out_files == []:
        raise "No .out files"
    train_out = out_files[0]
    
    with open(train_out) as f:
        _, _, _, Lasso, GroupLasso, Threshold = list(filter(None, model.split('_')))[:6]
        text = f.read().split()
        
        best_epoch = int(non_decimal.sub('', text[-7]))
        val_prec, val_rec, val_F = map(lambda x: float(non_decimal.sub('', x)), [text[-5], text[-3], text[-1]])
        
        results.append([model, Lasso, GroupLasso, Threshold, best_epoch, val_prec, val_rec, val_F])

In [53]:
results = pd.DataFrame(results, 
     columns=['folder', 'Lasso', 'GroupLasso', 'Threshold', 'best_epoch', 'Val_Prec', 'Val_Rec', 'Val_F1'])

In [54]:
results

,folder,Lasso,GroupLasso,Threshold,best_epoch,Val_Prec,Val_Rec,Val_F1
0,2020_04_07__0.1_0_-1__b718b076,0.1,0,-1,2,0.14359,0.14900,0.14625
1,2020_04_07__0.01_0_-1__53cfd74e,0.01,0,-1,2,0.13383,0.14941,0.14119
2,2020_04_07__0.001_0_-1__0f88a7ef,0.001,0,-1,45,0.44270,0.32493,0.37478
3,2020_04_07__0.0001_0_-1__92371c27,0.0001,0,-1,41,0.47256,0.36387,0.41115
4,2020_04_07__1e-05_0_-1__81cbb810,1e-05,0,-1,31,0.47419,0.36111,0.41000
5,2020_04_07__1e-06_0_-1__3866b871,1e-06,0,-1,31,0.46551,0.35087,0.40014
6,2020_04_07__0_0.1_-1__5810e829,0,0.1,-1,3,0.11740,0.15154,0.13230
7,2020_04_07__0_0.01_-1__b303bfa9,0,0.01,-1,24,0.38833,0.28967,0.33182
8,2020_04_07__0_0.001_-1__facca85d,0,0.001,-1,16,0.47305,0.35846,0.40786
9,2020_04_07__0_0.0001_-1__35a3d3de,0,0.0001,-1,48,0.47532,0.36839,0.41508


# Release

In [22]:
for index, row in results.iterrows():
    folder = results.iloc[index]['folder']
    epoch = results.iloc[index]['best_epoch']
    command = f'python code2seq.py --load models/java-small-model/{folder}/model_iter{epoch} --release'
    os.system(f'sbatch --error=./slurm/%j.err --output=./slurm/%j.out -J c2s_release --gres=gpu:1 -c 4 --wrap=\"{command}\"')

# Test

In [23]:
for index, row in results.iterrows():
    folder = results.iloc[index]['folder']
    epoch = results.iloc[index]['best_epoch']
    command = f'python3 code2seq.py --load {MODELS_PATH}/{folder}/model_iter{epoch}.release --test data/java-small/java-small.test.c2s'
    os.system(f'sbatch --error=./slurm/{folder}.err --output=./slurm/{folder}.out -J c2s_test --gres=gpu:1 -c 4 --wrap=\"{command}\"')


# Get test results

In [55]:
import pandas as pd

results['Test_Prec'] = None
results['Test_Rec'] = None
results['Test_F1'] = None

for index, row in results.iterrows():
    folder = results.iloc[index]['folder']
    epoch = results.iloc[index]['best_epoch']
    
    with open(f'./slurm/{folder}.out') as f:
        _, _, _, Lasso, GroupLasso, Threshold, _ = list(filter(None, folder.split('_')))
        text = f.read().split()
        prec_i = text.index('Precision:') + 1
        rec_i = text.index('recall:') + 1   
        f_i = text.index('F1:') + 1
        
        prec, rec, f = list(map(lambda a: float(a.rstrip(',')), [text[prec_i], text[rec_i], text[f_i]]))
        results['Test_Prec'][index] = prec
        results['Test_Rec'][index] = rec
        results['Test_F1'][index] = f

/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [56]:
results[['Lasso', 'GroupLasso', 'Threshold', 'best_epoch', 'Val_Prec',
       'Val_Rec', 'Val_F1', 'Test_Prec', 'Test_Rec', 'Test_F1']]

,Lasso,GroupLasso,Threshold,best_epoch,Val_Prec,Val_Rec,Val_F1,Test_Prec,Test_Rec,Test_F1
0,0.1,0,-1,2,0.14359,0.14900,0.14625,0.115612,0.0915634,0.102192
1,0.01,0,-1,2,0.13383,0.14941,0.14119,0.106646,0.091284,0.0983688
2,0.001,0,-1,45,0.44270,0.32493,0.37478,0.44021,0.324905,0.373869
3,0.0001,0,-1,41,0.47256,0.36387,0.41115,0.477011,0.367369,0.415071
4,1e-05,0,-1,31,0.47419,0.36111,0.41000,0.49157,0.377883,0.427293
5,1e-06,0,-1,31,0.46551,0.35087,0.40014,0.497002,0.373311,0.426367
6,0,0.1,-1,3,0.11740,0.15154,0.13230,0.106697,0.119965,0.112943
7,0,0.01,-1,24,0.38833,0.28967,0.33182,0.400704,0.28282,0.331597
8,0,0.001,-1,16,0.47305,0.35846,0.40786,0.477577,0.357234,0.408731
9,0,0.0001,-1,48,0.47532,0.36839,0.41508,0.487183,0.381508,0.427918


In [48]:
results.columns

Index(['folder', 'Lasso', 'GroupLasso', 'Threshold', 'best_epoch', 'Val_Prec',
       'Val_Rec', 'Val_F1', 'Test_Prec', 'Test_Rec', 'Test_F1'],
      dtype='object')

In [3]:
def get_logs(job, epoch):
    OUTPUT_FOLDER = './SLURM'
    
    
    
    with open(OUTPUT_FOLDER + f'{job}.out','r') as f:
        line = f.readline()
        while line != 'i'
        
    with open(OUTPUT_FOLDER + f'errors_{job}','r') as f:
        errors_file = f.read()




In [6]:
results

,jobID,lasso,group_lasso,threshold,epochs,Precision,recall,F1,NODES_VOCAB nonzeros,SUBTOKEN_VOCAB nonzeros,Lasso Reg,Group Lasso Reg
0,34508,0.0001,0.0001,1.000000e-03,24,0.47440,0.35912,0.40879,1236,30258,0.225216,0.065769
1,34510,0.0001,0.0001,1.000000e-05,64,0.46884,0.36772,0.41217,1486,74339,0.271612,0.061684
2,34511,0.0001,0.0001,1.000000e-06,75,0.46578,0.37260,0.41401,3986,148216,0.301428,0.062066
3,34512,0.0001,0.0001,1.000000e-07,24,0.46324,0.36765,0.40995,18028,6730523,0.408273,0.073424
4,34513,0.0000,0.0001,1.000000e-05,37,0.46248,0.36882,0.41037,40333,9413252,0.000000,4.115718
5,34514,0.0001,0.0000,1.000000e-05,60,0.47707,0.38251,0.42459,1434,73891,0.316947,0.000000


In [ ]:
python3 code2seq.py --load models/java-large-model/model_iter60.release --test data/java-small/java-small.test.c2s